### https://riptutorial.com/julia-lang/example/26313/guide<br>
`esc()` section

In [1]:
macro swap(p, q)
  quote
    tmp = $(esc(p))
    $(esc(p)) = $(esc(q))
    $(esc(q)) = tmp
  end
end

x,y = 1,2
@macroexpand @swap(x,y)

quote
    #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W0sZmlsZQ==.jl:3 =#
    var"#63#tmp" = x
    #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W0sZmlsZQ==.jl:4 =#
    x = y
    #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W0sZmlsZQ==.jl:5 =#
    y = var"#63#tmp"
end

In [2]:
@swap(x,y)
println(x,", ", y)  # 2 1

2, 1


`$` allows us to 'escape out of' the `quote`. So why not simply `$p` and `$q`? i.e.<br>
**`$` 允许我们“跳出” `quote` 。** 那么为什么不简单地 `$p` 和 `$q` 呢？即

In [3]:
macro swap2(p, q)
    quote
      tmp = $p
      $p = $q
      $q = tmp
    end
  end
  
  xx,yy = 11,22
  @macroexpand @swap2(xx,yy)

quote
    #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W3sZmlsZQ==.jl:3 =#
    var"#66#tmp" = var"#67#xx"
    #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W3sZmlsZQ==.jl:4 =#
    var"#67#xx" = var"#68#yy"
    #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W3sZmlsZQ==.jl:5 =#
    var"#68#yy" = var"#66#tmp"
end

In [4]:
@swap2(xx,yy)
println(xx,", ", yy) 

UndefVarError: UndefVarError: `#70#xx` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

Because that would look first to the `macro` scope for `p`, and it would find a local `p` i.e. the parameter `p` (yes, if you subsequently access `p` without `esc`-ing, the macro considers the `p` parameter as a local variable).<br>
因为这将首先在`macro`作用域查找`p`，并且它会找到一个局部 `p` ，即参数 `p` （是的，如果您随后访问 p 而没有 `esc`-ing，宏将 `p` 参数视为局部变量）。<br><br>

So `$p = ...` is just a assigning to the local `p`. it's not affecting whatever variable was passed-in in the calling context.<br>
所以 `$p = ...` 就是赋值给局部的 `p` 。它不会影响在调用上下文中传入的任何变量。

In [5]:
macro swap3(p, q)
    quote
      tmp = $p # you might think we don't need to esc() the RHS
      $(esc(p)) = $q
      $(esc(q)) = tmp
    end
  end
  
  xxx,yyy = 111,222
  @macroexpand @swap3(xxx,yyy)

quote
    #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W6sZmlsZQ==.jl:3 =#
    var"#72#tmp" = Main.xxx
    #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W6sZmlsZQ==.jl:4 =#
    xxx = Main.yyy
    #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W6sZmlsZQ==.jl:5 =#
    yyy = var"#72#tmp"
end

In [6]:
@swap3(xxx,yyy)
println(xxx,", ", yyy) 

222, 111


So `esc(p)` is 'leaking' `p` into the calling context. "The thing that was passed into the macro that we receive as `p`"<br>
因此 `esc(p)`  ‘泄漏’  `p` 进入调用上下文。“传入宏的东西，我们接收到 `p` ”<br><br>

As you can see tmp gets the **hygiene treatment** `#72#tmp`, whereas `xxx` and `yyy` don't. Julia is making a unique identifier for `tmp`, something you can manually do with `gensym`, ie:<br>
正如你所看到的， `tmp` 得到卫生处理 `#72#tmp` ，而 `xxx` 和 `yyy` 则没有。 Julia正在为 `tmp` 创建一个唯一的标识符，您可以手动使用 `gensym` ，即：

In [7]:
gensym(:tmp)

Symbol("##tmp#230")

In [8]:
module Swap
       export @swap4

       macro swap4(p, q)
         quote
           tmp = $p
           $(esc(p)) = $q
           $(esc(q)) = tmp
         end
       end
end

Main.Swap

In [9]:
using .Swap
x4,y4 = 1111,2222
@macroexpand @swap4(x4,y4)

quote
    #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X13sZmlsZQ==.jl:6 =#
    var"#74#tmp" = Main.Swap.x4
    #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X13sZmlsZQ==.jl:7 =#
    x4 = Main.Swap.y4
    #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X13sZmlsZQ==.jl:8 =#
    y4 = var"#74#tmp"
end

In [10]:
@swap4(x4,y4)
println(x4,", ", y4) 

UndefVarError: UndefVarError: `x4` not defined in `Main.Swap`
Suggestion: check for spelling errors or missing imports.

Another thing julia's macro hygiene does is, if the macro is from another module, it makes any variables (that were not assigned inside the macro's returning expression, like `tmp` in this case) globals of the current module, so `$p` becomes `Swap.$p`, likewise `$q` -> `Swap.$q`.<br>
julia的宏卫生做的另一件事是，如果宏来自另一个模块，它会使当前模块的任何变量（未在宏的返回表达式中分配，如 `tmp` 在这种情况下）成为全局变量，因此 `$p` 变为 `Swap.$p` ，同样 `$q`  ->  `Swap.$q` 。<br><br>

In general, if you need a variable that is outside the macro's scope you should esc it, so you should `esc(p)` and `esc(q)` regardless if they are on the LHS or RHS of a expression, or even by themselves.<br>
一般来说，如果你需要一个宏作用域之外的变量，你应该删除它，所以你应该 `esc(p)` 和 `esc(q)` ，**不管它们是在表达式的LHS还是RHS上**，甚至是它们自己。

In [11]:
module Swap2
       export @swap5

       macro swap5(p, q)
         quote
          tmp = $(esc(p))
          $(esc(p)) = $(esc(q))
          $(esc(q)) = tmp
         end
       end
end

Main.Swap2

In [12]:
using .Swap2
x5,y5 = 11111,22222
@macroexpand @swap5(x5,y5)

quote
    #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X20sZmlsZQ==.jl:6 =#
    var"#76#tmp" = x5
    #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X20sZmlsZQ==.jl:7 =#
    x5 = y5
    #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X20sZmlsZQ==.jl:8 =#
    y5 = var"#76#tmp"
end

In [13]:
@swap5(x5,y5)
println(x5,", ", y5) 

22222, 11111


In [14]:
macro until(condition, block)
    quote
        while ! $condition
            $block
        end
    end |> esc
end

i = 1
@macroexpand @until(i==5,  begin; print(i); i+=1; end)

quote
    #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X24sZmlsZQ==.jl:3 =#
    while !(i == 5)
        #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X24sZmlsZQ==.jl:4 =#
        begin
            #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X24sZmlsZQ==.jl:9 =#
            print(i)
            #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X24sZmlsZQ==.jl:9 =#
            i += 1
        end
        #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X24sZmlsZQ==.jl:5 =#
    end
end

In [15]:
i=1;  @until( i==5,  begin; print(i); i+=1; end )

1234

https://riptutorial.com/julia-lang/example/19404/until-loop<br>

Here we have used the function chaining syntax `|>`, which is equivalent to calling the `esc` function on the entire `quote` block. The `esc` function prevents macro hygiene from applying to the contents of the macro; without it, variables scoped in the macro will be renamed to prevent collisions with outside variables. See the Julia documentation on macro hygiene for more details.<br>

这里我们使用了函数链语法 `|>` ，这相当于在整个 `quote` 块上调用 `esc` 函数。 `esc` 函数防止宏卫生应用于宏的内容；没有它，宏中作用域的变量将被重命名，以防止与外部变量冲突。有关宏卫生的更多细节，请参阅Julia文档。<br>

You can use more than one expression in this loop, by simply putting everything inside a `begin ... end` block:<br>
你可以在这个循环中使用多个表达式，只需将所有内容放入 `begin ... end` 块中

`|>` is controversial, however. I am surprised a mob hasn't come to argue yet. (maybe everyone is just tired of it). There is a recommendation of having most if not all of the macro just be a call to a function, so:<br>
`|>` 是有争议的。我很惊讶还没有一群人来争论。（也许每个人都厌倦了）。有一个建议，如果不是所有的宏只是一个函数的调用，所以：

In [1]:
macro until(condition, block)
    show(block)
    esc(until(condition, block))
end

function until(condition, block)
    quote
        while !$condition
            $block
        end
    end
end

i = 1
@macroexpand @until(i==5,  begin; print(i); i+=1; end)

quote
    #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X30sZmlsZQ==.jl:15 =#
    print(i)
    #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X30sZmlsZQ==.jl:15 =#
    i += 1
end

quote
    #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X30sZmlsZQ==.jl:8 =#
    while !(i == 5)
        #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X30sZmlsZQ==.jl:9 =#
        begin
            #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X30sZmlsZQ==.jl:15 =#
            print(i)
            #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X30sZmlsZQ==.jl:15 =#
            i += 1
        end
        #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X30sZmlsZQ==.jl:10 =#
    end
end

In [17]:
i=1;  @until( i==5,  begin; print(i); i+=1; end )

1234

...is a safer alternative.<br>
...是一个更安全的选择。

Task: Swap the operands, so `swaps(1/2)` gives `2.00` i.e. `2/1`<br>
任务：交换操作数，因此 `swaps(1/2)` 给出 `2.00` 即 `2/1`

In [19]:
macro swaps(e)
    e.args[2:3] = e.args[3:-1:2]   
    e
end
@macroexpand @swaps(1/2)

:(2 / 1)

In [20]:
@swaps(1/2)

2.0

### https://riptutorial.com/julia-lang/example/26313/guide<br>
**A fun hack for using { } for blocks**<br>
**一个有趣的使用{}的代码块**<br><br>

**笔者注：忽略这个例子，没什么意义**

(@fcard) I don't think there is anything technical keeping `{}` from being used as blocks, in fact one can even pun on the residual `{}` syntax to make it work:<br>
（@fcard）我不认为有任何技术上的保持 `{}` 被用作块，事实上，人们甚至可以对残余的 `{}` 语法进行双关，以使其工作：

In [23]:
block_ex = quote 
print(1)
print(2)
1+2
end
Base.remove_linenums!(block_ex)

quote
    print(1)
    print(2)
    1 + 2
end

In [25]:
dump(block_ex)

Expr
  head: Symbol block
  args: Array{Any}((3,))
    1: Expr
      head: Symbol call
      args: Array{Any}((2,))
        1: Symbol print
        2: Int64 1
    2: Expr
      head: Symbol call
      args: Array{Any}((2,))
        1: Symbol print
        2: Int64 2
    3: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol +
        2: Int64 1
        3: Int64 2


In [31]:
macro c(block)
    @show(block.head)
    # @assert block.head == :cell1d
    esc(quote
        $(block.args...)
        end)
end

@macroexpand @c {
                 print(1)
                 print(2)
                 1+2
                }

block.head = :bracescat


quote
    #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X40sZmlsZQ==.jl:5 =#
    print(1)
    print(2)
    1 + 2
end

In [32]:
@c {
         print(1)
         print(2)
         1+2
       }

block.head = :bracescat
12

3

In [39]:
brace_ex = {
  print(1)
  print(2)
  1+2
}

ErrorException: syntax: { } matrix syntax is discontinued around e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X45sZmlsZQ==.jl:1

https://riptutorial.com/julia-lang/example/26313/guide<br>

**笔者注：这段markdown可忽略(没说明白),但示例可看**<br><br>
(@fcard) In this case because of lexical scope, a is undefined in `@M`s scope so it uses the global variable... I actually forgot to escape the flipplin' expression in my dumb example, but the "only works within the same module" part of it still applies.<br><br>
（@fcard）在这种情况下，由于词法作用域，a在` @M` 的作用域中是未定义的，所以它使用全局变量… 在我的愚蠢的例子中，我实际上忘记了转义flipplin'表达式，但它的“仅在同一模块内工作”部分仍然适用。

In [42]:
module M
       macro m()
         :(a+1)
       end
end

Main.M

In [43]:
using .M
a = 1
@macroexpand M.@m

:(Main.M.a + 1)

In [44]:
M.@m

UndefVarError: UndefVarError: `a` not defined in `Main.M`
Suggestion: check for spelling errors or missing imports.

In [45]:
@eval begin
    "do-until loop"
    macro $(:do)(block, until::Symbol, condition)
        until ≠ :until && 
            error("@do expected `until` got `$until`")
        quote
            let
                $block
                @until $condition begin
                    $block
                end
            end
        end |> esc
    end
end

@do

In [52]:
i = 0

0

https://riptutorial.com/julia-lang/example/26313/guide<br>
ADVANCED section

In [54]:
@macroexpand @do begin        
    # @show i      
    i += 1       
end until i == 5 

quote
    #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X54sZmlsZQ==.jl:7 =#
    let
        #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X54sZmlsZQ==.jl:8 =#
        begin
            #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X55sZmlsZQ==.jl:3 =#
            i += 1
        end
        #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X54sZmlsZQ==.jl:9 =#
        begin
            #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X30sZmlsZQ==.jl:7 =#
            while !(i == 5)
                #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X30sZmlsZQ==.jl:8 =#
                begin
                    #= e:\Projects.jl\Training.jl\10. macro\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X54sZmlsZQ==.jl:10 =#
                   

In [56]:
@do begin        
    @show i      
    i += 1       
end until i == 5

UndefVarError: UndefVarError: `i` not defined in local scope
Suggestion: check for an assignment to a local variable that shadows a global of the same name.

### https://docs.juliacn.com/latest/manual/metaprogramming/#卫生宏
**这个没看懂, 忽略也行**

In [ ]:
macro time(ex)
    show(ex)    
    
    return quote
        local t0 = time_ns()
        local val = $ex
        local t1 = time_ns()
        println("\nelapsed time: ", (t1-t0)/1e9, " seconds")
        val
    end
end

@time 3+2*5

:(3 + 2 * 5)
elapsed time: 3.9e-6 seconds


13

这种转义机制可以在必要时用于「违反」卫生，以便于引入或操作用户变量。例如，以下宏在其调用所处环境中将 x 设置为零：

In [1]:
macro zerox()
    return esc(:(x = 0))
end

function foo()
    x = 1
    @zerox
    return x # is zero
end

foo()

0

应当明智地使用这种变量操作，但它偶尔会很方便。<br>

获得正确的规则也许是个艰巨的挑战。**在使用宏之前，你可以去考虑是否函数闭包便已足够。另一个有用的策略是将尽可能多的工作推迟到运行时**。例如，许多宏只是将其参数封装为 `QuoteNode` 或类似的 `Expr`。这方面的例子有 `@task body`，它只返回 `schedule(Task(() -> $body))`， 和 `@eval expr`，它只返回 `eval(QuoteNode(expr))`。<br>

为了演示，我们可以将上面的 `@time` 示例重新编写成：

In [2]:
macro time(expr)
    return :(timeit(() -> $(esc(expr))))
end
function timeit(f)
    t0 = time_ns()
    val = f()
    t1 = time_ns()
    println("elapsed time: ", (t1-t0)/1e9, " seconds")
    return val
end

timeit (generic function with 1 method)

In [4]:
@time 3+2*5

elapsed time: 2.0e-7 seconds


13

但是，我们不这样做也是有充分理由的：将 `expr` 封装在新的作用域块（该匿名函数）中也会稍微改变该表达式的含义（其中任何变量的作用域），而我们想要 `@time` 使用时对其封装的代码影响最小。